# Converting County GeoJSON into Regular Bounding Box

## Part 1: Introduction

This Jupyter Notebook is intended to create county bounding box based on county GeoJSON/Shapefile.

## Part 2: Preparation

We will be using **Jupyter Notebook(anaconda 3)** to edit and run the script. Information on Anaconda installation can be found <a href='https://docs.anaconda.com/anaconda/install/'>here</a>. Please note that this script is running on Python 3.

***Usually, You can download county boundary GeoJSON file from state data portal, which typically is ArcGIS Hub.***

To run this script you need:
- county GeoJSON/Shapefile stored in **state** folder
- directory path (**geojson** folder > **state** folder)

The script currently prints one GeoJSON file:
- **state_County_bbox.json**

>Original created on Jan 31 2021<br>
@author: Yijing Zhou @YijingZhou33

## Part 3: Get Started

###  Step 1: Import modules

In [1]:
import pandas as pd
import os
import geopandas as gpd
import json
from itertools import chain
import string
import folium

### Step 2: Manual items to change

In [2]:
state = 'Delaware'

rawdata = os.path.join('geojson', state, state + '_County_boundaries.json')
output = os.path.join('geojson', state, state + '_County_bbox.json')

## Part 4: Build up county GeoJSON schema

###  Step 3: Convert GeoJSON to JSON

In [3]:
def conversion(rawdata):
    inputfile = gpd.read_file(rawdata)
    ## convert file to json 
    inputfile = json.loads(inputfile.to_json())
    ## display features properties as dataframe
    df = pd.json_normalize(inputfile['features'])
    return df

df = conversion(rawdata)

###  Step 4: Create bounding box

In [4]:
def bbox(points):
    x_coordinates, y_coordinates = zip(*points)
    return ','.join(str(x) for x in [min(x_coordinates), min(y_coordinates), max(x_coordinates), max(y_coordinates)])

def coordinates_bbox(df):
    for _, row in df.iterrows():
        ## geometry is Polygon
        if type(row['geometry.coordinates'][0][0][0]) is float:
            row['geometry.coordinates'] = bbox(row['geometry.coordinates'][0])
        else:
        ## geometry is Multipolygon
            row['geometry.coordinates'] = bbox(list(chain.from_iterable([l[0] for l in row['geometry.coordinates']])))
            
coordinates_bbox(df)

###  Step 6: Round coordinates to 4 decimal places

In [5]:
## create regular bouding box coordinate pairs and round them to 4 decimal places
df = pd.concat([df, df['geometry.coordinates'].str.split(',', expand=True).astype(float)], axis=1).rename(
    columns={0:'minX', 1:'minY', 2:'maxX', 3:'maxY'})
df['maxXmaxY'] = df.apply(lambda row: [row.maxX, row.maxY], axis = 1)
df['maxXminY'] = df.apply(lambda row: [row.maxX, row.minY], axis = 1)
df['minXminY'] = df.apply(lambda row: [row.minX, row.minY], axis = 1)
df['minXmaxY'] = df.apply(lambda row: [row.minX, row.maxY], axis = 1)
df['geometry.coordinates'] = df[['maxXmaxY', 'maxXminY', 'minXminY', 'minXmaxY', 'maxXmaxY']].values.tolist()

## clean up unnecessary columns
df_clean = df.drop(columns =['minX', 'minY', 'maxX', 'maxY', 'maxXmaxY', 'maxXminY', 'minXminY', 'minXmaxY'])

## Part 5: Create County GeoJSON

###  Step 7: Create geojson features

In [6]:
# create_geojson_features 
def create_geojson_features(df):
    print('> Creating GeoJSON features...')
    features = []
    geojson = {
        'type': 'FeatureCollection',
        'features': features
    }
    for _, row in df.iterrows():
        feature = {
            'type': 'Feature',
            'geometry': {
                'type':'Polygon', 
                'coordinates':[row['geometry.coordinates']]
            },
            'properties': {
                'County': row['properties.County'],
                'State': row['properties.State']
            }
        }

        features.append(feature)
    return geojson

data_geojson = create_geojson_features(df_clean)

> Creating GeoJSON features...


###  Step 8: Generate geojson file

In [7]:
with open(output, 'w') as txtfile:
    json.dump(data_geojson, txtfile)
print('> Creating GeoJSON file...')

> Creating GeoJSON file...


## Part 6: Inspect bounding box map

In [8]:
print('> Making map...')
## change the location here to zoom to the center
m = folium.Map(location = [42.3756, -93.6397], control_scale = True, zoom_start = 5)

## check if the indexmap geojson files can be rendered properly
folium.GeoJson(output, 
               tooltip = folium.GeoJsonTooltip(fields=('County', 'State'),
               aliases=('County', 'State')),
               show = True).add_to(m)
m

> Making map...
